# Detection and Quantification of Microspots

---

**Updated on:** 2023-09-11

> Authors: Simon Knoblauch (simon.knoblauch@student.uni-tuebingen.de)  
> Input file format: .tif
> Output file formats: None currently  
> Dependencies: imageio, pandas, numpy, skimage, matplotlib, ipympl  

**Please read the comments before proceeding to run the code and let us know if you run into any errors or find undocumented pieces of code.**

In [ ]:
# Importing dependencies
import imageio.v3 as iio
import pandas as pd
import skimage
import matplotlib.pyplot as plt
import numpy as np
import ipympl
%matplotlib widget